In [1]:
from osrs_fcns import *
import pandas as pd
import numpy as np
from datetime import datetime
from time import perf_counter_ns

In [2]:
def getItemInfo(item):
    if(type(item) == str):
        mapping = getLatestItemMap(idFirst=False)
        itemID = mapping[item]
        itemInfo = cleanItemDF(formatItemDF(queryDB(f"""SELECT * from items WHERE item_id = {itemID} LIMIT 1""")))
    else:
        itemID = item
        itemInfo = cleanItemDF(formatItemDF(queryDB(f"""SELECT * from items WHERE item_id = {itemID} LIMIT 1""")))
        
    return itemInfo

In [3]:
getItemInfo('Battlestaff')

item_id  members  lowalch  buyLimit   value  highalch             icon  \
0     1391     True   2800.0   11000.0  7000.0    4200.0  Battlestaff.png   

          name                         examine  
0  Battlestaff  It's a slightly magical stick.

In [4]:
def getItemPrices(items,dateBegin,dateEnd):
    t1_start = perf_counter_ns()    
    #convert dateStr to dtObj and then to seconds
    dateStart = convertDateToTimestamp(dateBegin)
    dateEnd = convertDateToTimestamp(dateEnd)
    t_dtconvert = perf_counter_ns()
    print('Time to convert dates:',(t_dtconvert-t1_start)/1000000000)
    
    #convert each itemName to item_id
    print('Mapping Names To IDs')
    mapping = getLatestItemMap(idFirst=False)
    item_ids = []
    for i in items:
        item_ids.append(mapping[i])
    t_mapping = perf_counter_ns()
    print('Time to map item names:',(t_mapping-t_dtconvert)/1000000000)
    
    print('Querying DB')
    fullDF = cleanPriceDF(formatPriceDF(queryDB(f"""SELECT * from prices WHERE timestamp > {dateStart} AND timestamp < {dateEnd}""")))
    t_query = perf_counter_ns()
    print('Time to query DB:',(t_query-t_mapping)/1000000000)

    print('Filtering DB')
    filteredDF = fullDF.loc[fullDF['item_id'].isin(item_ids)]
    t_filter = perf_counter_ns()
    print('Time to filter DF:',(t_filter-t_query)/1000000000)
    return filteredDF

In [5]:
def convertDateToTimestamp(dateStr):
    dtObj = datetime.strptime(dateStr, '%d/%m/%y %H:%M:%S').timestamp()
    return dtObj

In [6]:
items = getItemPrices(['Battlestaff','Bandos tassets'],'03/08/21 00:00:00', '03/09/21 00:00:00')

Time to convert dates: 0.0006512
Mapping Names To IDs
Time to map item names: 0.1788652
Querying DB
Time to query DB: 83.8997485
Filtering DB
Time to filter DF: 0.0391958


In [7]:
def resampleDF(df,resampleStr):
    return df.resample(resampleStr).mean()



In [8]:
resampleDF(items,'6h')

item_id  avgHighPrice  highPriceVolume   avgLowPrice  \
timestamp                                                                       
2021-08-03 00:00:00  6612.500000  1.180019e+07       677.909091  1.124379e+07   
2021-08-03 06:00:00  6612.500000  1.218093e+07       461.798611  1.120987e+07   
2021-08-03 12:00:00  6612.500000  1.157637e+07      1121.979167  1.114608e+07   
2021-08-03 18:00:00  6612.500000  1.282025e+07       472.986111  1.117378e+07   
2021-08-04 00:00:00  6612.500000  1.145321e+07      1474.888889  1.119049e+07   
...                          ...           ...              ...           ...   
2021-09-02 00:00:00  6612.500000  1.094444e+07      1458.152778  1.091774e+07   
2021-09-02 06:00:00  6575.986014  1.033176e+07       985.300699  1.071359e+07   
2021-09-02 12:00:00  6612.500000  1.088540e+07      1596.631944  1.108919e+07   
2021-09-02 18:00:00  6612.500000  1.139088e+07      1736.312500  1.103500e+07   
2021-09-03 00:00:00  6612.500000  1.118854e+07      1425.925000  1.106335e+07   

                     lowPriceVolume  
timestamp                            
2021-08-03 00:00:00     2130.818182  
2021-08-03 06:00:00     2380.986111  
2021-08-03 12:00:00     3114.937500  
2021-08-03 18:00:00     3938.708333  
2021-08-04 00:00:00     5150.423611  
...                             ...  
2021-09-02 00:00:00     4004.027778  
2021-09-02 06:00:00     2053.216783  
2021-09-02 12:00:00     3091.645833  
2021-09-02 18:00:00     3493.305556  
2021-09-03 00:00:00     3611.408333  

[125 rows x 5 columns]